In [1]:
# code to scrap halodoc articles from api (json) and loop each page

import requests
from bs4 import BeautifulSoup
import pandas as pd

base_url = 'https://magneto.api.halodoc.com/api/cms/articles?page_no='
results_url = []
results_title = []
results_content = []

# try 1 - 1000
# loop each page
for page in range(1, 1000):
    # get url for each page and make request
    url = base_url + str(page)
    response = requests.get(url)
    data = response.json()

    # Loop through articles
    for article in data['result']:

        # Get URL for article
        article_url = article['source_url']

        # Make request to article URL
        article_response = requests.get(article_url)

        # parsing data
        soup = BeautifulSoup(article_response.text, features="html.parser")

        # get element by class
        container = soup.find(attrs={'class': 'article-page__article-body-wrapper'})
        if(container is None):
            print(f'{article_url} dilewati (variable container kosong) - page {page}')
            continue

        # get titles
        titles = container.findAll(attrs={'class': 'section-header__content-text-title'})

        # get content
        contents = container.findAll(attrs={'class': 'article-page__article-body'})

        # append
        results_url.append(article_url)
        for title in titles:
            results_title.append(title.text)
        for content in contents:
            results_content.append(content.text)



https://www.halodoc.com/ketahui-5-jenis-masker-dan-fungsinya-untuk-wajah dilewati (variable container kosong) - page 21
https://www.halodoc.com/sering-diabaikan-ini-5-hal-yang-sering-memicu-depresi-pada-remaja dilewati (variable container kosong) - page 267
https://www.halodoc.com/inilah-kiat-mudah-memberikan-asi-saat-sudah-masuk-kerja dilewati (variable container kosong) - page 701
https://www.halodoc.com/perempuan-berusia-lanjut-ini-cara-menjaga-kesehatan-tubuh dilewati (variable container kosong) - page 846


In [2]:
# create dataframe
df = pd.DataFrame({
                   'url': results_url, 
                   'title': results_title, 
                   'title_length' : [len(title) for title in results_title],
                   'content': results_content, 
                   'content_count' : [len(content) for content in results_content]
                   })

# save to csv with good format
df.to_csv('halodoc1-1000.csv', index=False, encoding='utf-8-sig')

In [3]:
# read halodoc.csv to dataframe

df = pd.read_csv('halodoc1-1000.csv')
df

,url,title,title_length,content,content_count
0,https://www.halodoc.com/ketahui-segala-hal-men...,Ketahui Segala Hal Mengenai COVID-19,38,“Infeksi COVID-19 adalah salah satu penyakit y...,6532
1,https://www.halodoc.com/bagaimana-cara-merawat...,Bagaimana Cara Merawat Karies Gigi?,37,“Ada banyak pilihan perawatan medis dan rumaha...,5993
2,https://www.halodoc.com/cegah-burnout-ini-tand...,"Cegah Burnout, Ini Tanda Pekerja Butuh Piknik...",61,“Burnout merupakan kondisi ketika seseorang m...,4982
3,https://www.halodoc.com/kenalan-dengan-terapi-...,Kenalan dengan Terapi ABA untuk Anak Autis,44,“Salah satu cara untuk mengurangi gejala auti...,4855
4,https://www.halodoc.com/10-herbal-untuk-mereda...,10 Herbal untuk Meredakan Sakit Perut Bagian ...,52,“Ada banyak herbal yang dapat membantu mereda...,3638
...,...,...,...,...,...
9981,https://www.halodoc.com/sebelum-membeli-kacama...,"Sebelum Membeli Kacamata Minus, Perhatikan Ha...",52,\n“Kacamata adalah salah satu alat bantu yang ...,4745
9982,https://www.halodoc.com/cek-fakta-peran-kakek-...,Cek Fakta: Peran Kakek Nenek dalam Menjaga Ke...,66,\n“Kedekatan kakek nenek dengan cucu dapat mem...,4204
9983,https://www.halodoc.com/waspada-ketahui-dampak...,"Waspada, Ketahui Dampak Tubuh Kekurangan Vita...",52,"“Kurangnya asupan vitamin, termasuk vitamin D...",5744
9984,https://www.halodoc.com/ketahui-4-olahraga-unt...,Ketahui 4 Olahraga untuk Ibu Hamil Selama di ...,52,"\n“Saat ibu hamil harus di rumah saja, bukan b...",4360
